## Divisão do Dataframe


In [14]:
import pandas as pd 
import os
from math import ceil

### Carregamento do Dataframe Principal

In [ ]:
path = "C:\\Users\\edinocencio\\DataClenupCmsCrm\\DataSources\\PMD-CMS.xlsx"
sheet_name="CEPS"
df = pd.read_excel(path, sheet_name="CEPS")

df.head(1)

,CEP_OLD,NUM_CARACT_OLD,CEP_NEW,NUM_CARACT_NEW
0,01331-010,9,01331-010,9


### Selecionar e limpar coluna de CEP

In [15]:
df = df[['CEP_NEW']]
df['CEP_NEW'] = df['CEP_NEW'].str.replace('-','').str.strip()
df.drop_duplicates(inplace=True)

In [16]:
# Determinar o número de linhas por arquivo (por exemplo, 100 linhas por arquivo)
lines_per_file = 20

### Calcular o número de arquivos necessários

In [17]:
lines_total = len(df)
number_of_files = ceil(lines_total / lines_per_file)

In [18]:
# Criar diretório para salvar os arquivos divididos
output_dir = 'DataSources\\Divididos'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Dividir e salvar os arquivos

for i in range(number_of_files):
    start = i * lines_per_file
    end = start + lines_per_file
    # print("i = " + str(i) + " lines = "+ str(lines_per_file) + " start = " + str(start) + " end = " + str(end))
    df_divided =  df.iloc[start:end]
    file_name = os.path.join(output_dir, f'file_{i + 1}.xlsx')
    df_divided.to_excel(file_name,index=False)

# Buscar o CEP com base na função desenvolvida

In [ ]:
from utils.functions import find_cep
import glob
from time import sleep

# Diretório de arquivos divididos
from urllib import request
import requests

In [ ]:
def find_cep(cep, cache):
    if cep in cache:
        return cache[cep]
    
    else:
    # URL_VIA_CEP = f'https://viacep.com.br/ws/{cep}/json/'
        URL_VIA_CEP = f'https://opencep.com/v1/{cep}'
        try:
            answer = requests.get(URL_VIA_CEP, timeout=5)
            if answer.ok:
                address_ = answer.json()
                if "erro" in address_:
                    cache[cep] = {
                        "cep":"",
                        "logradouro":"",
                        "complemento":"",
                        "bairro":"",
                        "localidade": "",
                        "uf":"",
                        "ibge":"",
                        "erro":"INVALID CEP",
                    }
                else:
                    cache[cep]={
                        "cep": address_.get("cep",""),
                        "logradouro":address_.get("logradouro",""),
                        "complemento":address_.get("complemento",""),
                        "bairro":address_.get("bairro",""),
                        "localidade":address_.get("localidade",""),
                        "uf":address_.get("uf",""),
                        "ibge":address_.get("ibge",""),
                        "erro":"",
                    }
            else:
                cache[cep] = {
                        "cep":"",
                        "logradouro":"",
                        "complemento":"",
                        "bairro":"",
                        "localidade": "",
                        "uf":"",
                        "ibge":"",
                        "erro":f"Erro HTTP {answer.status_code}",
                    }
        except Exception as e:
            cache[cep] = {
                "cep":"",
                        "logradouro":"",
                        "complemento":"",
                        "bairro":"",
                        "localidade": "",
                        "uf":"",
                        "ibge":"",
                "erro": "Erro de conexão",
            }
        
        # Aguardar para evitar sobrecarregar a API
        return cache[cep]
        sleep(0.4) #900 ms    

In [ ]:

input_dir = 'DataSources\\Divididos\\'
output_dir = 'DataSources\\Completos\\'


i = 1
files = glob.glob(f'{input_dir}*.xlsx')

cache = {}
enderecos = []

for file in files:
    df = pd.read_excel(file, dtype={"CEP_NEW": str})
    # Inicializar cache e lista para armazenar os endereços
    # Processar cada CEP
    for cep in df['CEP_NEW']:
        endereco = find_cep(cep,cache=cache)
        enderecos.append(endereco)

    # Criar DataFrame dos resultados de consulta
    enderecos_df = pd.DataFrame(enderecos)
    # Concatenar os resultados ao DataFrame original
    df_completo = pd.concat([df.reset_index(drop=True), enderecos_df], axis=1)
    df_completo.to_excel("DataSources\\revisados\\cep_" + str(i) + ".xlsx")
    enderecos = []
    i+=1



### Grouping Files

In [1]:
import os
import pandas as pd

output_dir = 'DataSources\\revisados\\'
DIR_FILES = os.listdir(output_dir)

df_final = pd.DataFrame()

for file in DIR_FILES:
    filepath = output_dir + file
    df = pd.read_excel(filepath, dtype={"CEP_NEW": str})
    df = df.iloc[:,1:]
    df_final= pd.concat([df_final,df])

output_dir = 'DataSources\\finalizados\\'
file_name = 'base_cep_consolidada.xlsx'
full_file_path = output_dir + file_name

# df_cep_errors = df_final[df_final['erro'].notnull()]
df_final = df_final[df_final['erro'].isnull()]

# df_final.to_excel(full_file_path, index=False)
# df_cep_errors.to_excel(output_dir + 'base_cep_erros.xlsx', index=False)

In [2]:
path_database_master = 'C:\\Users\\edinocencio\\DataClenupCmsCrm\\DataSources\\PMD-CMS.xlsx'

df_master = pd.read_excel(path_database_master, sheet_name='CEPS')
df_master = df_master.merge(df_final,left_on='CEP_NEW', right_on='cep')
df_master = df_master[['CEP_OLD','CEP_NEW_x','logradouro', 'complemento','bairro','localidade','uf']]

In [3]:
df_master = df_master.rename(columns={
                            'CEP_OLD':'cep_antigo',
                            'CEP_NEW_x':'cep_validado',
                            'localidade':'cidade'
                        })

In [4]:
# Dicionário para mapear siglas de UF para os nomes dos estados corretos
uf_to_estado = {
    'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas', 'BA': 'Bahia',
    'CE': 'Ceará', 'DF': 'Distrito Federal', 'ES': 'Espírito Santo', 'GO': 'Goiás',
    'MA': 'Maranhão', 'MT': 'Mato Grosso', 'MS': 'Mato Grosso do Sul', 'MG': 'Minas Gerais',
    'PA': 'Pará', 'PB': 'Paraíba', 'PR': 'Paraná', 'PE': 'Pernambuco', 'PI': 'Piauí',
    'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte', 'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia', 'RR': 'Roraima', 'SC': 'Santa Catarina', 'SP': 'São Paulo',
    'SE': 'Sergipe', 'TO': 'Tocantins'
}

# Atualizar a coluna 'Estado' com os nomes corretos baseados na coluna 'UF'
df_master['Estado'] = df_master['uf'].map(uf_to_estado)
df_master.to_excel(full_file_path, index=False)